# 引用包

In [8]:
import functools
import os
import json
import re
import torch
from tqdm import tqdm
from torch_geometric.data import *
import networkx as nx

import hashlib
def stringtomd5(originstr):
    originstr = originstr.encode("utf-8")
    signaturemd5 = hashlib.sha256()
    signaturemd5.update(originstr)
    return signaturemd5.hexdigest()
filePath="./ta1-theia-e3-official/"

filelist = os.listdir(filePath)

In [5]:
filelist

['ta1-theia-e3-official-6r.json.9',
 'ta1-theia-e3-official-1r.json.9',
 'ta1-theia-e3-official-6r.json.8',
 'ta1-theia-e3-official-6r.json.x12',
 'ta1-theia-e3-official-1r.json.7',
 'ta1-theia-e3-official-6r.json.7',
 'ta1-theia-e3-official-6r.json.5',
 'ta1-theia-e3-official-1r.json.3',
 'ta1-theia-e3-official-6r.json.0',
 'ta1-theia-e3-official-1r.json.5',
 'ta1-theia-e3-official-6r.json.x11',
 'ta1-theia-e3-official-1r.json.4',
 'ta1-theia-e3-official-1r.json.6',
 'ta1-theia-e3-official-5m.json.0',
 'ta1-theia-e3-official-1r.json.2',
 'ta1-theia-e3-official-6r.json.x10',
 'ta1-theia-e3-official-6r.json.4',
 'ta1-theia-e3-official-6r.json.1',
 'ta1-theia-e3-official-3.json.0',
 'ta1-theia-e3-official-1r.json.8',
 'ta1-theia-e3-official-1r.json.1',
 'ta1-theia-e3-official-6r.json.6',
 'ta1-theia-e3-official-6r.json.2',
 'ta1-theia-e3-official-6r.json.3',
 'ta1-theia-e3-official-1r.json.0']

In [6]:
# 时间戳转换函数

from datetime import datetime, timezone
import time
import pytz
from time import mktime
from datetime import datetime
import time
def ns_time_to_datetime(ns):
    """
    :param ns: int 纳秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00.000000000
    """
    dt = datetime.fromtimestamp(int(ns) // 1000000000)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s


def ns_time_to_datetime_US(ns):
    """
    :param ns: int 纳秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00.000000000
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(ns) // 1000000000, tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s

def time_to_datetime_US(s):
    """
    :param ns: int 秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(s), tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    
    return s

def datetime_to_ns_time(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    timeStamp = int(time.mktime(timeArray))
    timeStamp = timeStamp * 1000000000
    return timeStamp

def datetime_to_ns_time_US(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp=timestamp.timestamp()
    timeStamp = timestamp * 1000000000
    return int(timeStamp)

def datetime_to_timestamp_US(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp=timestamp.timestamp()
    timeStamp = timestamp
    return int(timeStamp)


# 数据库

## 连接数据库

In [11]:
# 连接postgessql 数据库，将数据存入数据库中
import psycopg2

from psycopg2 import extras as ex
connect = psycopg2.connect(database = 'tc_theia3_share_db',
                           user = 'postgres',
                           password = 'Ssys8.21',
                           port = '5432'#一般是5432
                          )
# 创建一个cursor来执行数据库的操作
cur = connect.cursor()

In [52]:
tt=cur.execute("""
    delete from subject_node_table where 1=1;
""")
print(tt)
connect.commit()  #需要手动提交

None


In [53]:
#发生错误时需要回滚
connect.rollback()

## 数据库表

create table event_table
(
    src_node      varchar,
    src_index_id  varchar,
    operation     varchar,
    dst_node      varchar,
    dst_index_id  varchar,
    timestamp_rec bigint,
    _id           serial
);

alter table event_table
    owner to postgres;

create unique index event_table__id_uindex
    on event_table;

grant delete, insert, references, select, trigger, truncate, update on event_table to psql;



create table file_node_table
(
    node_uuid varchar not null,
    hash_id   varchar not null,
    path      varchar,
    constraint file_node_table_pk
        primary key (node_uuid, hash_id)
);

alter table file_node_table
    owner to psql;



create table netflow_node_table
(
    node_uuid varchar not null,
    hash_id   varchar not null,
    src_addr  varchar,
    src_port  varchar,
    dst_addr  varchar,
    dst_port  varchar,
    constraint netflow_node_table_pk
        primary key (node_uuid, hash_id)
);

alter table netflow_node_table
    owner to psql;



create table node2id
(
    hash_id   varchar                                                    not null
        constraint node2id_pk
            primary key,
    node_type varchar,
    msg       varchar,
    index_id  integer default nextval('node2id_index_id_seq1'::regclass) not null
);

alter table node2id
    owner to psql;

create unique index node2id_hash_id_uindex
    on node2id (hash_id);



create table subject_node_table
(
    node_uuid varchar not null,
    hash_id   varchar not null,
    "cmdLine" varchar not null,
    tgid      varchar not null,
    path      varchar not null,
    constraint subject_node_table_pk
        primary key (node_uuid, hash_id)
);

alter table subject_node_table
    owner to psql;



# 分别处理不同类型节点

## 套接字节点

In [9]:
# 遍历数据集
netobjset=set()
netobj2hash={}# 
datalist=[]
for file in tqdm(filelist):
        with open(filePath + file, "r") as f:
            for line in f:
#                 pass
                if '{"datum":{"com.bbn.tc.schema.avro.cdm18.NetFlowObject"' in line :
            
                    try:
                        res=re.findall('NetFlowObject":{"uuid":"(.*?)"(.*?)"localAddress":"(.*?)","localPort":(.*?),"remoteAddress":"(.*?)","remotePort":(.*?),',line)[0]

                        nodeid=res[0]
                        srcaddr=res[2]
                        srcport=res[3]
                        dstaddr=res[4]
                        dstport=res[5]

                        nodeproperty=srcaddr+","+srcport+","+dstaddr+","+dstport # 只关注向哪里发起的网络流 合理么？
                        hashstr=stringtomd5(nodeproperty)
                        netobj2hash[nodeid]=[hashstr,nodeproperty]
                        netobj2hash[hashstr]=nodeid
                        netobjset.add(hashstr)
                    except:
                        
                        pass




100%|██████████| 25/25 [02:44<00:00,  6.58s/it]


In [12]:
datalist=[]
for i in netobj2hash.keys():
    if len(i)!=64:
        datalist.append([i]+[netobj2hash[i][0]]+netobj2hash[i][1].split(","))

#写入数据库


sql = '''insert into netflow_node_table
                     values %s
        '''
ex.execute_values(cur,sql, datalist,page_size=10000)
connect.commit()  #需要手动提交

## 进程节点

In [46]:
scusess_count=0
fail_count=0
subject_obj2hash={}# 

subjectset=set()
subject2hash={}

datalist=[]
for file in tqdm(filelist):
        with open(filePath + file, "r") as f:
            for line in f:
                # 转JSON  需要try 包起来，有的不是标准的json串
                if '{"datum":{"com.bbn.tc.schema.avro.cdm18.Subject"' in line :
                    res=re.findall('Subject":{"uuid":"(.*?)"(.*?)"cmdLine":{"string":"(.*?)"}(.*?)"properties":{"map":{"tgid":"(.*?)"',line)[0]
                    try:
                        path_str=re.findall('"path":"(.*?)"',line)[0] 
                        path=path_str
                    except:
                        path="null"
                    nodeid=res[0]
                    cmdLine=res[2]
                    tgid=res[4]
                    
                    nodeproperty=cmdLine+","+tgid+","+path
                    hashstr=stringtomd5(nodeproperty)
                    subject2hash[nodeid]=[hashstr,cmdLine,tgid,path]
                    subject2hash[hashstr]=nodeid
                    subjectset.add(hashstr)
                        

100%|██████████| 25/25 [02:42<00:00,  6.50s/it]


In [31]:
subject2hash

{}

In [47]:

datalist=[]
for i in subject2hash.keys():
    if len(i)!=64:
        datalist.append([i]+subject2hash[i])


In [48]:
datalist[0][2]

'/usr/lib/postgresql/9.1/bin/postgres -D /var/lib/postgresql/9.1/main -c config_file=/etc/postgresql/9.1/main/postgresql.conf'

In [39]:
i

'4631A522-0000-0000-0000-000000000020'

In [49]:
len(datalist)

279369

In [54]:
#写入数据库


sql = '''insert into subject_node_table
                     values %s
        '''
ex.execute_values(cur,sql, datalist,page_size=10000)
connect.commit()  #需要手动提交      

## 文件节点

In [55]:
# 构建文件节点的唯一标识 多个uuid 对一个hash值，  多对一的关系
fileset=set()
file2hash={}# 丢弃了 绝对路径为空的实体，  那么在构建边的时候，遇到不存在的key 就应该跳过当前的event了
nullcount=0


for file in tqdm(filelist):
        with open(filePath + file, "r") as f:
            for line in f:
                # 转JSON  需要try 包起来，有的不是标准的json串
                if '{"datum":{"com.bbn.tc.schema.avro.cdm18.FileObject"' in line :
                    try:

                        res=re.findall('FileObject":{"uuid":"(.*?)"(.*?)"filename":"(.*?)"',line)[0]
                        nodeid=res[0]
                        filepath=res[2]
                        nodeproperty=filepath
                        hashstr=stringtomd5(nodeproperty)
                        file2hash[nodeid]=[hashstr,nodeproperty]
                        file2hash[hashstr]=nodeid
                        fileset.add(hashstr)
                    except:
                        # 228786个文件节点是没有路径的
                        pass

100%|██████████| 25/25 [03:44<00:00,  9.00s/it]


In [56]:
datalist=[]
for i in file2hash.keys():
    if len(i)!=64:
        datalist.append([i]+file2hash[i])

In [57]:
# 写入数据库
sql = '''insert into file_node_table
                     values %s
        '''
ex.execute_values(cur,sql, datalist,page_size=10000)
connect.commit()  #需要手动提交

## 处理事件数据

In [58]:

# 生成node2id表的数据
node_list={}
##################################################################################################
sql="""
select * from file_node_table;
"""
cur.execute(sql)
records = cur.fetchall()

for i in records:    
    node_list[i[1]]=["file",i[-1]]

file_uuid2hash={}
for i in records:
    file_uuid2hash[i[0]]=i[1]
##################################################################################################    
sql="""
select * from subject_node_table;
"""
cur.execute(sql)
records = cur.fetchall()

for i in records:
    node_list[i[1]]=["subject",i[-1]]

subject_uuid2hash={}
for i in records:
    subject_uuid2hash[i[0]]=i[1]
##################################################################################################
sql="""
select * from netflow_node_table;
"""
cur.execute(sql)
records = cur.fetchall()

for i in records:
    
    node_list[i[1]]=["netflow",i[-2]+":"+i[-1]]

net_uuid2hash={}
for i in records:
    net_uuid2hash[i[0]]=i[1]

In [59]:
node_list_database=[]
node_index=0
for i in node_list:
    node_list_database.append([i]+node_list[i]+[node_index])
    node_index+=1

In [61]:
#写入数据库
sql = '''insert into node2id
                     values %s
        '''
ex.execute_values(cur,sql, node_list_database,page_size=10000)
connect.commit()  #需要手动提交

In [62]:
# 构建nodeid 与msg之间的构建  创建字典变量 nodeid2msg
sql="select * from node2id ORDER BY index_id;"
cur.execute(sql)
rows = cur.fetchall()

nodeid2msg={}  # 可以实现 nodeid 转 msg      node hash 转 nodeid
for i in rows:
    nodeid2msg[i[0]]=i[-1]
    nodeid2msg[i[-1]]={i[1]:i[2]}  #0-828297

In [64]:
nodeid2msg

{'17ce196318e17c2c5c2fbbb631e7de266ab192597eba89487682492e8c77cb6f': 0,
 0: {'file': '/run/shm/org.chromium.wLJAmM'},
 '164157bba7655bd5906ba831ca6d081d94422778ca59f31b6e8a661d2d0aa5fb': 1,
 1: {'file': '/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/FDD330928F58D4E68CA52036B088F9BB90C78234'},
 '7339881adeaf47450a5b7fd27997213fa5303b6f718fcae17d2c46f73b4a2678': 2,
 2: {'file': '/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/703B0116FA24C9B10FCABCEFB8C0A9347E7C04B4'},
 'ceff08aeff578a051a1bfc429d5cfebb293710f171f69ca6405f34b7850e5ce2': 3,
 3: {'file': '/run/shm/org.chromium.VOfHVG'},
 'b06bc1ede93bdb38ab447a13b1e0be8caff9a7fa411d5bb4c62ef47564c7d429': 4,
 4: {'file': '/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/2553421D84357120AEA23C275AE8E9D23EB76AFE'},
 'd4dc1666d3ba8bf7f28747dbd7ca10b00c6a449e902d52685ac2a9fdaf179377': 5,
 5: {'file': '/home/admin/.cache/mozilla/firefox/pe11scpa.default/cache2/entries/9288DCCF0A3E7371451B

In [67]:


datalist=[]


# firsttime=1522764134000000000

for file in tqdm(filelist):
        with open(filePath + file, "r") as f:
            for line in f:
                if '{"datum":{"com.bbn.tc.schema.avro.cdm18.Event"' in line and "EVENT_FLOWS_TO" not in line:
                    time=re.findall('"timestampNanos":(.*?),',line)[0]
                    time=int(time) # 将时间转成秒为单位
                    subjectid=re.findall('"subject":{"com.bbn.tc.schema.avro.cdm18.UUID":"(.*?)"',line)[0]
                    objectid=re.findall('"predicateObject":{"com.bbn.tc.schema.avro.cdm18.UUID":"(.*?)"',line)[0]
                    relation_type=re.findall('"type":"(.*?)"',line)[0]


                    if subjectid in subject2hash.keys():
                        # 该实体是存在的
                        subjectid=subject2hash[subjectid][0]# 将uuid转成节点的hash值
                    if objectid in subject2hash.keys():
                        objectid=subject2hash[objectid][0]# 将uuid转成节点的hash值
                    if objectid in file2hash.keys():
                        objectid=file2hash[objectid][0]# 将uuid转成节点的hash值
                    if objectid in netobj2hash.keys():
                        objectid=netobj2hash[objectid][0]# 将uuid转成节点的hash值
                    if len(subjectid)==64 and len(objectid) == 64: # uuid都转成hashid才能被认作成为一条合法的事件 能够作为关系存下来
                        if relation_type in ['EVENT_READ','EVENT_READ_SOCKET_PARAMS','EVENT_RECVFROM','EVENT_RECVMSG']:
                            # 数据流入到subject中，因此process作为流入节点
                            datalist.append((objectid,nodeid2msg[objectid],relation_type,subjectid,nodeid2msg[subjectid],time))
                        else:
                            datalist.append((subjectid,nodeid2msg[subjectid],relation_type,objectid,nodeid2msg[objectid],time))

                        # 写入数据库       
                        
                        if len(datalist)>=10000:
                            try:                
                                sql = '''insert into event_table
                                                     values %s
                                        '''
                                ex.execute_values(cur,sql, datalist,page_size=100000)
                                connect.commit()  #需要手动提交
                                datalist=[]
                            except:
                                print("数据库写入失败！")
                                connect.rollback()

100%|██████████| 25/25 [57:18<00:00, 137.53s/it]


# 属性编码

In [68]:
# 生成数据集之前，首先算出所有node 和edge的特征映射字典，这样数据集生成速度会加快


from sklearn.feature_extraction import FeatureHasher
from torch_geometric.transforms import NormalizeFeatures

from sklearn import preprocessing #导入用于数据标准化的模块
import numpy as np
# 将path 转成分层的list

FH_string=FeatureHasher(n_features=16,input_type="string")
FH_dict=FeatureHasher(n_features=16,input_type="dict")


def path2higlist(p):
    l=[]
    spl=p.strip().split('/')
    for i in spl:
        if len(l)!=0:
            l.append(l[-1]+'/'+i)
        else:
            l.append(i)
#     print(l)
    return l

def ip2higlist(p):
    l=[]
    spl=p.strip().split('.')
    for i in spl:
        if len(l)!=0:
            l.append(l[-1]+'.'+i)
        else:
            l.append(i)
#     print(l)
    return l
def list2str(l):
    s=''
    for i in l:
        s+=i
    return s

In [69]:
node_msg_vec=[]
node_msg_dic_list=[]
for i in tqdm(nodeid2msg.keys()):
    if type(i)==int:
        if 'netflow' in nodeid2msg[i].keys():
            higlist=['netflow']
            higlist+=ip2higlist(nodeid2msg[i]['netflow'])
            
        if 'file' in nodeid2msg[i].keys():
            higlist=['file']
            higlist+=path2higlist(nodeid2msg[i]['file'])
            
#             print(higlist)
        if 'subject' in nodeid2msg[i].keys():
            higlist=['subject']
            higlist+=path2higlist(nodeid2msg[i]['subject'])
        node_msg_dic_list.append(list2str(higlist))

100%|██████████| 1656624/1656624 [00:04<00:00, 402445.65it/s]


In [70]:
node2higvec=[]
for i in tqdm(node_msg_dic_list):
    vec=FH_string.transform([i]).toarray()
    node2higvec.append(vec)


100%|██████████| 828312/828312 [02:02<00:00, 6747.37it/s]


In [71]:
node2higvec=np.array(node2higvec).reshape([-1,16])

In [73]:
rel2id={1: 'EVENT_CONNECT',
 'EVENT_CONNECT': 1,
 2: 'EVENT_EXECUTE',
 'EVENT_EXECUTE': 2,
 3: 'EVENT_OPEN',
 'EVENT_OPEN': 3,
 4: 'EVENT_READ',
 'EVENT_READ': 4,
 5: 'EVENT_RECVFROM',
 'EVENT_RECVFROM': 5,
 6: 'EVENT_RECVMSG',
 'EVENT_RECVMSG': 6,
 7: 'EVENT_SENDMSG',
 'EVENT_SENDMSG': 7,
 8: 'EVENT_SENDTO',
 'EVENT_SENDTO': 8,
 9: 'EVENT_WRITE',
 'EVENT_WRITE': 9}

In [74]:
# 生成edge type one-hot变量
relvec=torch.nn.functional.one_hot(torch.arange(0, len(rel2id.keys())//2), num_classes=len(rel2id.keys())//2)

In [75]:
# 将不同的relation type转为 变量 rel2vec   通过该变量，可以将字符串的关系类型转成ont hot编码
rel2vec={}
for i in rel2id.keys():
    if type(i) is not int:
        rel2vec[i]= relvec[rel2id[i]-1]
        rel2vec[relvec[rel2id[i]-1]]=i

In [76]:
## 保存结果变量
torch.save(node2higvec,"node2higvec")
torch.save(rel2vec,"rel2vec")

# 生成数据集

In [77]:
# 加载节点特征字典，边特征字典数据
node2higvec=torch.load("./node2higvec")
rel2vec=torch.load("./rel2vec")

In [ ]:
for day in tqdm(range(2,14)):
    start_timestamp=datetime_to_ns_time_US('2018-04-'+str(day)+' 00:00:00')
    end_timestamp=datetime_to_ns_time_US('2018-04-'+str(day+1)+' 00:00:00')
    sql="""
    select * from event_table
    where
          timestamp_rec>'%s' and timestamp_rec<'%s'
           ORDER BY timestamp_rec;
    """%(start_timestamp,end_timestamp)
    cur.execute(sql)
    events = cur.fetchall()
    print('2018-04-'+str(day)," events count:",str(len(events)))
    edge_list=[]
    for e in events:
        edge_temp=[int(e[1]),int(e[4]),e[2],e[5]]
        if e[2] in rel2id:
            edge_list.append(edge_temp)
    print('2018-04-'+str(day)," edge list len:",str(len(edge_list)))
    # 保存数据集数据
    dataset = TemporalData()
    src = []
    dst = []
    msg = []
    t = []
    for i in edge_list:
        src.append(int(i[0]))
        dst.append(int(i[1]))
    #     msg.append(torch.cat([torch.from_numpy(node2higvec_bn[i[0]]), rel2vec[i[2]], torch.from_numpy(node2higvec_bn[i[1]])] ))
        msg.append(torch.cat([torch.from_numpy(node2higvec[i[0]]), rel2vec[i[2]], torch.from_numpy(node2higvec[i[1]])] ))
        t.append(int(i[3]))

    dataset.src = torch.tensor(src)
    dataset.dst = torch.tensor(dst)
    dataset.t = torch.tensor(t)
    dataset.msg = torch.vstack(msg)
    dataset.src = dataset.src.to(torch.long)
    dataset.dst = dataset.dst.to(torch.long)
    dataset.msg = dataset.msg.to(torch.float)
    dataset.t = dataset.t.to(torch.long)
    torch.save(dataset, "./train_graph/graph_4_"+str(day)+".TemporalData.simple")  